In [10]:
import dotenv
import os
from langchain_openai import OpenAI

dotenv.load_dotenv()

True

In [11]:
SECRET_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(api_key=SECRET_KEY, temperature=0.7)

# First test


In [12]:
explanation = llm.invoke(
    "Using the latest information, provide a buy or sell recommendation for the stock of Apple Inc. (AAPL). Add reasoning for your recommendation."
)
explanation


"\n\nBased on the latest information, I would recommend a buy for the stock of Apple Inc. (AAPL). My reasoning for this recommendation is as follows:\n\n1. Strong Financial Performance: Apple recently reported its Q3 earnings, which exceeded analyst expectations. The company posted revenue of $59.7 billion, an increase of 11% year-over-year, and a net income of $11.25 billion, an increase of 12.9% year-over-year. This strong financial performance indicates that the company is continuing to grow and generate profits, which is a positive sign for investors.\n\n2. Diversified Product Portfolio: Apple has a diversified product portfolio, including the iPhone, iPad, Mac, and wearables like the Apple Watch. This diversification reduces the company's reliance on any single product and helps to mitigate risk. Additionally, the company has been expanding into new areas, such as services and streaming, which could drive future growth.\n\n3. Upcoming Product Launches: Apple is expected to launch 

# Adding a prompt template


In [13]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["stock"],
    input_types={
        "stock": str,
    },
    template="Using the latest information, provide a buy or sell recommendation for the stock of {stock}. Add reasoning for your recommendation.",
)

prompt.format(stock="Amazon Inc. (AMZN)")

'Using the latest information, provide a buy or sell recommendation for the stock of Amazon Inc. (AMZN). Add reasoning for your recommendation.'

# Making this as a chain


In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

chain.invoke(input={"stock": "Amazon Inc. (AMZN)"})

{'stock': 'Amazon Inc. (AMZN)',
 'text': "\n\nBased on the latest information, I would recommend a buy for the stock of Amazon Inc. (AMZN).\n\nOne of the main reasons for this recommendation is Amazon's strong financial performance. In the second quarter of 2021, Amazon reported net sales of $113.1 billion, which is a 27% increase from the same period last year. The company also saw a significant increase in operating income, which grew by 31% to $7.7 billion.\n\nAdditionally, Amazon's Prime membership program continues to be a major source of revenue and customer retention for the company. The company reported that Prime Day, its annual shopping event exclusive to Prime members, was its biggest sales event ever, surpassing sales from Black Friday and Cyber Monday combined.\n\nMoreover, Amazon's expansion into new markets, such as healthcare and advertising, provides further growth opportunities for the company. In the healthcare sector, Amazon launched its telehealth service, Amazon C

# Let's try to add a search function


In [15]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

tool.run("Obama's first name?")

"1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE ... OBAMA, II. Day. 4. 6b. Island. Year. 5b. Hour. 1961 7:24 P.M.. His last name, Obama, was derived from his Luo descent. Apr 2, 2018 ... What's Barack Obama's nickname? · His full name is Barack Hussein Obama II. · I have only heard him called by these\xa0... First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President, Barack Obama. She is the first\xa0... Dec 19, 2018 ... His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name\xa0... Barack Hussein Obama II was born August 4, 1961, in Honolulu, Hawaii, to parents Barack H. Obama, Sr., and Stanley Ann Dunham. Jan 19, 2017 ... Hopeful parents named their sons for the first Black president, whose name is a variation of the Hebrew name Baruch\xa0... When Barack Obama was elected president in 2008, he became the first African American to hold the office. The framers of t

# Steps for providing a solution

1. Ask about the stock and provide context
2. Search financial information, news and stock price
3. Provide a summary of the stock
4. Based on the information, provide a recommendation


In [29]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain.tools import BaseTool

llm = OpenAI(api_key=SECRET_KEY, temperature=0.7)


class SearchTool(BaseTool):
    name = "Google Search"
    description = "Search Google for recent results. Input is a string."

    def _run(self, query: str):
        search = GoogleSearchAPIWrapper()
        return search.run(query)

    def _arun(self, query: str):
        raise NotImplementedError


tools = [SearchTool()]

# Create an agent
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm=llm, prompt=prompt, tools=tools)

prompt


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [28]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

executor.invoke(input={"input": "BYD"})




> Entering new AgentExecutor chain...
 Search BYD on Google
Action: Google Search
Action Input: BYDBYD has developed blade battery, electronic platform 3.0 and dual-mode hybrid technology for electric cars, giving full play to the advantages of ... byd-energy-prod-new. byd-skyrail-prod-new. Rail Transit · SkyRail · SkyShuttle · Batteries · Electronics · SkyRail · SkyShuttle. BYD Europe HQS Scorpius 112, 2132 LR Hoofddorp, Netherlands. BYD Auto ... BYD Auto Co., Ltd. is the automotive subsidiary of BYD Company, a publicly listed Chinese multinational manufacturing company. It manufactures ... The BYD Battery-Box Premium LVL is a lithium iron phosphate (LFP) battery for use with an external inverter. Thanks to its control and communication port (BMU), ... Mar 2, 2023 ... BYD | 225091 followers on LinkedIn. Build Your Dreams. | Established in 1995, BYD is a top high-tech enterprise in China specializing in IT, ... Boyd Gaming (BYD) Expected to Beat Earnings Estimates: Can the Stock Move

{'input': 'BYD',
 'output': 'BYD is a Chinese multinational manufacturing company that specializes in IT, rail transit, electric cars, batteries, electronics, and more. They are known for their advanced technologies and have a presence in various industries worldwide. They also own Boyd Gaming, a company that operates hotels, casinos, and restaurants across the US.'}